In [1]:
# Install required packages
!pip install -qU transformers datasets wandb accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/arabic-history-questions-dataset/all_data_flattened.json
/kaggle/input/arabic-history-questions-dataset/all_data.json
/kaggle/input/arabic-history-questions-dataset/train.json
/kaggle/input/arabic-history-questions-dataset/test.json
/kaggle/input/arabic-history-questions-dataset/test_raw.json
/kaggle/input/arabic-history-questions-dataset/val_raw.json
/kaggle/input/arabic-history-questions-dataset/train_raw.json
/kaggle/input/arabic-history-questions-dataset/val.json


In [3]:
# Import libraries
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForSeq2SeqLM, 
    AutoTokenizer, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments, 
    DataCollatorForSeq2Seq
)
from datasets import DatasetDict, Dataset
# from huggingface_hub import notebook_login
import wandb

2025-06-21 02:00:06.075514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750471206.288088      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750471206.346576      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# # Login to Hugging Face Hub
# notebook_login()

wandb.login(key='77eafacbf29d3f89b810de78fe1f766a9b6e6fe8')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: noureenaboarab2003 (noureenaboarab2003-ain-shams-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
# Load dataset function
def load_data(train_path, val_path):
    print("🔄 Loading datasets...")
    train_data = pd.read_json(train_path)
    val_data = pd.read_json(val_path)
    
    train_data.rename(columns={"paragraph": "context"}, inplace=True)
    val_data.rename(columns={"paragraph": "context"}, inplace=True)
    
    print("\n📊 Training Data Sample:")
    print(train_data.head(1).T)
    print("\n📊 Validation Data Sample:")
    print(val_data.head(1).T)
    
    return train_data, val_data

train_path = "/kaggle/input/arabic-history-questions-dataset/train.json"
val_path = "/kaggle/input/arabic-history-questions-dataset/val.json"

train_data, val_data = load_data(train_path, val_path)

🔄 Loading datasets...

📊 Training Data Sample:
                                                          0
context   ـ مساكن الطبقة الوسطى: كانت مخصصة للفنيين من ا...
question  ما هو الغرض الرئيسي من الممر الذي يتفرع إلى به...
answer                                   للاستقبال والترفيه

📊 Validation Data Sample:
                                                          0
context   كانت الحكومة المصرية على عهد محمد على حكومة مط...
question         ما الفرق بين حكومة محمد علي وعصر المماليك؟
answer                                 وجود نظام إداري منظم


In [6]:
# Format dataset
def format_for_qg(dataset, dataset_name="train"):
    print(f"\n🔍 Formatting {dataset_name} dataset...")
    formatted_text =dataset["context"]
    
    print("\n📝 Sample Formatted Input (first 200 chars):")
    for i in range(3):
        print(f"Sample {i+1}: {formatted_text.iloc[i][:200]}...")
    
    return pd.DataFrame({
        "text": formatted_text,
        "question": dataset["question"]
    })

train_qg = format_for_qg(train_data, "train")
val_qg = format_for_qg(val_data, "validation")


🔍 Formatting train dataset...

📝 Sample Formatted Input (first 200 chars):
Sample 1: ـ مساكن الطبقة الوسطى: كانت مخصصة للفنيين من الصناع، وكانت أكثر اتساعًا، ويشمل كل منزل فناء مكشوفًا وأربع حجرات أبوابها مفتحة عليها، وتتصل بخمس حجرات أخرى، وكانت الحجرات جميعها مسقوفه بقوائم )عروق(من ...
Sample 2: القصر الفرعونى: تميز القصر الفرعونى بالاتساع والفخامة، وكان يسمى بـ)البيت العظيم، ( وقُسم إلى قسمين: قسم عام يستقبل فيه الأشراف وكبار رجال الدولة، وقسم خاص يشمل القاعات الخاصة، وكانت تعتمد على أسقف أع...
Sample 3: توفي إمبراطور قوانغشو في ١٤ نوفمبر ١٩٠٨ ، وفي ١٥ نوفمبر ١٩٠٨ ، توفي تسيشي أيضا . وزعمت الشائعات أنها أو يوان شيكاي أمرت الخصيان الموثوق بهم بتسميم إمبراطور قوانغشو ، وأكد تشريح الجثة الذي أجري بعد ما ...

🔍 Formatting validation dataset...

📝 Sample Formatted Input (first 200 chars):
Sample 1: كانت الحكومة المصرية على عهد محمد على حكومة مطلقة تسود فىها قاعدة حكم الفرد، ولكن الفارق بينها وبين ما كانت عليه فى عصر المماليك أن محمد على باشا وضع نظاما لإدارتها، فحل هذا النظام محل الفوضى

In [7]:
# Convert to DatasetDict
datasets_qg = DatasetDict({
    "train": Dataset.from_pandas(train_qg),
    "validation": Dataset.from_pandas(val_qg)
})

In [8]:
datasets_qg

DatasetDict({
    train: Dataset({
        features: ['text', 'question'],
        num_rows: 4702
    })
    validation: Dataset({
        features: ['text', 'question'],
        num_rows: 550
    })
})

In [9]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration


# # Load the model and tokenizer
# model_name = "google/flan-t5-base"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)


# # Add <sep> token
# special_tokens = {'additional_special_tokens': ['<sep>']}
# tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))

# # Verify tokenization
# sample_text = "السؤال: متى تأسست السعودية؟ <sep> الجواب: 1932 <sep> السياق: تأسست المملكة..."
# print("\n🔍 Tokenization Test:")
# print("Tokens:", tokenizer.tokenize(sample_text)[:20])
# print("<sep> token ID:", tokenizer.convert_tokens_to_ids("<sep>"))

In [10]:
# # Load AraBART-finetuned-wiki-ar
# print("\n🔄 Loading AraBART model...")
# model_name = "moussaKam/AraBART"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "moussaKam/AraBART"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Try loading with safetensors first (if available)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_safetensors=True)


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

In [11]:

printed_samples = 0  # Global variable

# Tokenization function optimized for AraBART
def tokenize_function(examples):
    global printed_samples  # Access the global variable
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["question"],
            max_length=128,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
    
    model_inputs["labels"] = labels["input_ids"]
    
# Print first 5 samples
    for i in range(len(model_inputs["input_ids"])):
        if printed_samples < 5:
            print(f"\n🔢 Sample {printed_samples + 1}:")
            print("Input:", tokenizer.decode(model_inputs["input_ids"][i][:20]))
            print("Label:", tokenizer.decode(labels["input_ids"][i][:10]))
            printed_samples += 1
        else:
            break
    
    return model_inputs

tokenized_qg_datasets = datasets_qg.map(tokenize_function,
                                  batched=True,
                                  batch_size=8,
                                  remove_columns=["text", "question"])

Map:   0%|          | 0/4702 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(



🔢 Sample 1:
Input: <s> <unk> مساكن الطبقة الوسطى: كانت مخصصة للفنيين من الصناع، وكانت أكثر اتساع<unk>ا
Label: <s> ما هو الغرض الرئيسي من الممر الذي يتفرع

🔢 Sample 2:
Input: <s> القصر الفرعونى: تميز القصر الفرعونى بالاتساع والفخامة، وكان يسمى ب<unk>
Label: <s> أين كانت تقع غرفة النوم في القصر الفرعوني

🔢 Sample 3:
Input: <s> توفي إمبراطور قوانغشو في ١٤ نوفمبر ١٩٠٨ ، وفي ١٥ نوفمبر ١٩٠٨ ، توفي تسي
Label: <s> ما الذي جعل الحكومة التي أنشأها زايف

🔢 Sample 4:
Input: <s> ظهر الدور العلمى للمسجد مع بداية تأسيس النبي صلى الله عليه وسلم لمسجد المدينة المنورة، وقد تطور
Label: <s> ما هي الأهداف التعليمية للكتاتيب في

🔢 Sample 5:
Input: <s> كانت الأسرة الغنية تستأجر مرضعات لأطفالهم، ويبعثون بأبنائهم إلى مدرسة المعبد
Label: <s> ما هي المواد الدراسية التي كان يدرسها الابن


Map:   0%|          | 0/550 [00:00<?, ? examples/s]

In [12]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, GenerationConfig

# # model_name = "UBC-NLP/AraT5-base"
# model_name = "Abdou/arabic-tashkeel-flan-t5-small"

# tokenizer = AutoTokenizer.from_pretrained(model_name)

# data_collator = DataCollatorForSeq2Seq(
#     tokenizer=tokenizer,
#     padding=True,  # Dynamically pad to the longest in the batch
#     label_pad_token_id=-100  # Ensures padded tokens in labels don't affect training
# )

# def tokenize_function(examples):
#     # return tokenizer(
#     #     examples["text"],  # Input: Context + Answer
#     #     text_target=examples["required"],
#     #     # padding="max_length",
#     #     padding= True,
#     #     truncation=True,
#     #     label_pad_token_id=-100 
#     # )
#     model_inputs = tokenizer(
#         examples["text"],  # Input: Context
#         max_length=768,  # Limit input length
#         truncation=True,
#         padding="max_length",  # Ensures uniform input size
#     )

#     labels = tokenizer(
#         examples["required"],  # Output: Target
#         max_length=100,  # Target length is much shorter
#         truncation=True,
#         padding="max_length"  # Ensures uniform output size
#     )

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# tokenized_qg_datasets = datasets_qg.map(tokenize_function, batched=True, remove_columns=["text", "required"])
# # tokenized_ag_datasets = datasets_ag.map(tokenize_function, batched=True, remove_columns=["text", "required"])


In [13]:
print(type(tokenized_qg_datasets))  # Should be <class 'datasets.dataset_dict.DatasetDict'>
print(tokenized_qg_datasets)  # Prints dataset details
print(tokenized_qg_datasets["train"][0])


<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4702
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 550
    })
})
{'input_ids': [0, 31, 3, 16808, 5664, 3326, 21, 99, 7932, 24656, 496, 6, 48, 17252, 4, 454, 128, 18301, 3, 12, 4, 14652, 52, 1940, 39363, 37824, 3, 12, 39308, 2673, 22, 19767, 71, 2329, 13, 166, 4, 149, 17675, 27205, 2673, 22, 181, 4, 454, 3736, 22, 9792, 71, 38584, 9, 35, 26722, 85, 422, 14029, 10722, 420, 8831, 4, 454, 12409, 130, 6563, 26, 8915, 13, 4, 49862, 949, 2673, 22, 1079, 44, 9760, 63, 1487, 4, 454, 6563, 10, 29656, 1479, 9238, 6, 8831, 4, 454, 9515, 183, 16966, 56, 52, 11249, 29, 71, 37760, 29, 5373, 27, 56, 9775, 21434, 901, 3, 12, 4, 193, 13860, 452, 11249, 7610, 5192, 7610, 11890, 4, 193, 145, 33191, 7932, 54, 15046, 4, 187, 1820, 3, 12, 37542, 19, 33191, 7932, 7189, 8793, 4, 103, 28509,

In [14]:
# Function to decode and print input and labels
def print_decoded_example(dataset, split, index):
    print(f"\nExample from {split} split (index {index}):")
    print("Decoded input:", tokenizer.decode(dataset[split][index]["input_ids"]))
    print("Decoded labels:", tokenizer.decode(dataset[split][index]["labels"]))
    print("-" * 50)

# Print examples from train, validation, and test splits
print_decoded_example(tokenized_qg_datasets, "train", 200)  # First example from training set
print_decoded_example(tokenized_qg_datasets, "validation", 2)  # First example from validation set



Example from train split (index 200):
Decoded input: <s> وقد تميز فن النحت الرومانى بمميزات عديدة أهمها: ١. الواقعية الشديدة: مثل رأس قيصر المصنوع من حجر البازلت وتمثال بومبى. وتماثيل أغسطس. ٢. الابتكار والتجديد: وخاصة<unk> فى نحت التماثيل الشخصية وكذلك الفنون المعمارية ومنها: معبد البانثييون، وأقواس النصر والأعمدة التذكارية.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [15]:
# Function to check special tokens
def check_special_tokens(tokenizer):
    print("\nSpecial tokens:")
    print(f"<context>: {tokenizer.convert_tokens_to_ids('<context>')}")
    print(f"<answer>: {tokenizer.convert_tokens_to_ids('<answer>')}")
    print(f"<question>: {tokenizer.convert_tokens_to_ids('<question>')}")
    print(f"</s>: {tokenizer.convert_tokens_to_ids('</s>')}")

In [16]:
# Define special tokens
special_tokens_dict = {"additional_special_tokens": ["<context>", "<answer>", "<question>"]}

# Add them to the tokenizer
tokenizer.add_special_tokens(special_tokens_dict)

# Check again
check_special_tokens(tokenizer)



Special tokens:
<context>: 50002
<answer>: 50003
<question>: 50004
</s>: 2


In [17]:
# Check for NaN or inf in the dataset
import numpy as np

def check_for_invalid_values(dataset):
    for split in dataset:
        print(f"Checking {split} split...")
        for example in dataset[split]:
            if np.isnan(example["input_ids"]).any() or np.isinf(example["input_ids"]).any():
                print(f"Invalid input_ids found in {split} split!")
            if np.isnan(example["labels"]).any() or np.isinf(example["labels"]).any():
                print(f"Invalid labels found in {split} split!")

check_for_invalid_values(tokenized_qg_datasets)

Checking train split...
Checking validation split...


In [18]:
# Check special tokens
print("Special tokens:")
print(f"<context>: {tokenizer.convert_tokens_to_ids('<context>')}")
print(f"<answer>: {tokenizer.convert_tokens_to_ids('<answer>')}")
print(f"<question>: {tokenizer.convert_tokens_to_ids('<question>')}")
print(f"</s>: {tokenizer.convert_tokens_to_ids('</s>')}")

# Decode a few examples to verify tokenization
for i in range(3):
    print(f"\nExample {i + 1}:")
    print("Input IDs:", tokenized_qg_datasets["train"][i]["input_ids"])
    print("Decoded Input:", tokenizer.decode(tokenized_qg_datasets["train"][i]["input_ids"]))
    print("Labels:", tokenized_qg_datasets["train"][i]["labels"])
    print("Decoded Labels:", tokenizer.decode(tokenized_qg_datasets["train"][i]["labels"]))

Special tokens:
<context>: 50002
<answer>: 50003
<question>: 50004
</s>: 2

Example 1:
Input IDs: [0, 31, 3, 16808, 5664, 3326, 21, 99, 7932, 24656, 496, 6, 48, 17252, 4, 454, 128, 18301, 3, 12, 4, 14652, 52, 1940, 39363, 37824, 3, 12, 39308, 2673, 22, 19767, 71, 2329, 13, 166, 4, 149, 17675, 27205, 2673, 22, 181, 4, 454, 3736, 22, 9792, 71, 38584, 9, 35, 26722, 85, 422, 14029, 10722, 420, 8831, 4, 454, 12409, 130, 6563, 26, 8915, 13, 4, 49862, 949, 2673, 22, 1079, 44, 9760, 63, 1487, 4, 454, 6563, 10, 29656, 1479, 9238, 6, 8831, 4, 454, 9515, 183, 16966, 56, 52, 11249, 29, 71, 37760, 29, 5373, 27, 56, 9775, 21434, 901, 3, 12, 4, 193, 13860, 452, 11249, 7610, 5192, 7610, 11890, 4, 193, 145, 33191, 7932, 54, 15046, 4, 187, 1820, 3, 12, 37542, 19, 33191, 7932, 7189, 8793, 4, 103, 28509, 528, 6688, 20654, 101, 37542, 19, 115, 97, 7224, 5835, 9, 13590, 4842, 2693, 6737, 19015, 3459, 16506, 13, 4, 454, 33191, 1795, 1367, 711, 116, 20713, 63, 10, 45840, 4, 30593, 22893, 12348, 3227, 7, 2, 1,

In [19]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    label_pad_token_id=-100
)

In [20]:
import torch
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim import AdamW
import math

# Calculate total training steps correctly
dataset_size = len(tokenized_qg_datasets["train"])
batch_size = 4  # per_device_train_batch_size
grad_accum_steps = 8  # gradient_accumulation_steps
num_epochs = 15

# Effective batch size per step
effective_batch_size = batch_size * grad_accum_steps

# Steps per epoch (use ceiling to ensure we don't underestimate)
steps_per_epoch = math.ceil(dataset_size / effective_batch_size)

# Total steps across all epochs
total_steps = steps_per_epoch * num_epochs

print(f"Dataset size: {dataset_size}")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Total steps: {total_steps}")

training_args = Seq2SeqTrainingArguments(
    output_dir="./finetuned-AraT5-QG",
    eval_strategy="epoch",
    learning_rate=3e-5,  # This will be the max learning rate for OneCycleLR
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    fp16=True,
    generation_num_beams=3,
    gradient_accumulation_steps=8,
    save_strategy="epoch",
    report_to="none",
    load_best_model_at_end=True,
    # Add these for better control
    logging_steps=50,  # Log every 50 steps
    eval_steps=steps_per_epoch,  # Evaluate every epoch
    save_steps=steps_per_epoch,  # Save every epoch
    dataloader_drop_last=False,  # Don't drop incomplete batches
)

# Create optimizer
optimizer = AdamW(
    model.parameters(),
    lr=training_args.learning_rate,
    weight_decay=training_args.weight_decay
)

# Create OneCycleLR scheduler with correct total_steps
scheduler = OneCycleLR(
    optimizer,
    max_lr=training_args.learning_rate,
    total_steps=total_steps,
    pct_start=0.1,  # Percentage of cycle spent increasing learning rate
    anneal_strategy='cos',  # Cosine annealing
    cycle_momentum=True,
    base_momentum=0.85,
    max_momentum=0.95,
    div_factor=25.0,  # Initial learning rate = max_lr / div_factor
    final_div_factor=1e4  # Final learning rate = initial_lr / final_div_factor
)

# Trainer with custom optimizer and scheduler
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_qg_datasets["train"],
    eval_dataset=tokenized_qg_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler)  # Pass custom optimizer and scheduler
)

import gc

# Clean up memory before training
gc.collect()
torch.cuda.empty_cache()

# Start training
try:
    trainer.train()
except Exception as e:
    print(f"Training error: {e}")
    print(f"Current step: {trainer.state.global_step}")
    print(f"Expected total steps: {total_steps}")
    raise

Dataset size: 4702
Steps per epoch: 147
Total steps: 2205


/tmp/ipykernel_20/1003783964.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,13.085100,8.607812
2,8.776200,0.263493
3,0.263500,0.186405
4,0.204200,0.176120
5,0.171700,0.173483
6,0.157900,0.173011
7,0.144400,0.173669
8,0.137900,0.174794
9,0.128000,0.175507
10,0.121000,0.176972


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will ins

In [21]:
# # Training arguments optimized for AraBART
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./arabart-qg",
#     eval_strategy="epoch",
#     learning_rate=3e-5,
#     per_device_train_batch_size=4,  # Reduced for memory
#     per_device_eval_batch_size=4,
#     weight_decay=0.01,
#     save_total_limit=2,
#     num_train_epochs=10,
#     predict_with_generate=True,
#     fp16=False,
#     logging_steps=50,
#     save_strategy="epoch",
#     generation_num_beams=4,
#     report_to="wandb",
# )

# # Initialize trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_qg_datasets["train"],
#     eval_dataset=tokenized_qg_datasets["validation"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

In [22]:
# # Start training
# trainer.train()

In [23]:
model.save_pretrained("./arabart_finetuned/final_model")
tokenizer.save_pretrained("./arabart_finetuned/final_model")

('./arabart_finetuned/final_model/tokenizer_config.json',
 './arabart_finetuned/final_model/special_tokens_map.json',
 './arabart_finetuned/final_model/sentencepiece.bpe.model',
 './arabart_finetuned/final_model/added_tokens.json',
 './arabart_finetuned/final_model/tokenizer.json')

In [24]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./finetuned-AraT5-QG",
#     evaluation_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     predict_with_generate=True,
#     weight_decay=0.01,
#     save_total_limit=2,
#     num_train_epochs=5,
#     fp16=False,  # Disable mixed precision for debugging
#     generation_num_beams=3,
#     gradient_accumulation_steps=4,
#     # gradient_checkpointing=True,
#     save_strategy="epoch",
#     logging_dir="./logs",
# )
# # print(training_args)  # Verify the training arguments


# # Trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_qg_datasets["train"],
#     eval_dataset=tokenized_qg_datasets["validation"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# # Start training
# trainer.train()